In [5]:
# Load packages
import time
import numpy as np
import pandas as pd
from numba import jit,float64
from scipy.optimize import minimize

In [6]:
# Load data
data = pd.read_csv('UnitaryData.csv')
pd_lag = np.array(data['d.p'])

# Calculate terciles for pd ratio
tercile_1 = np.quantile(pd_lag,1./3)
tercile_2 = np.quantile(pd_lag,2./3)

# Calculate indicator based on today's pd ratio
pd_lag_indicator = np.array([pd_lag <= tercile_1,(pd_lag <= tercile_2) & (pd_lag > tercile_1),pd_lag > tercile_2]).T

# Calculate indicator for tomorrow's pd ratio
pd_indicator = pd_lag_indicator[1:]

# Drop last row since we do not have tomorrow's pd ratio at that point
pd_lag_indicator = pd_lag_indicator[:-1]
X = np.array(data[['Rf','Rm-Rf','SMB','HML']])[:-1]
f = np.hstack((X * pd_lag_indicator[:,:1],X * pd_lag_indicator[:,1:2],X * pd_lag_indicator[:,2:3]))
g = np.array(data['log.RW'])[:-1]

## 8 Intertemporal Divergence Constraints

### Proposition 8.6
Problem 8.4 can be solved by finding the solution to:

\begin{equation}
\epsilon = \min_\hat{\lambda}\mathbb E \left(\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}\cdot f(X_1)\right]\left( \frac{e_1}{e_0}\right) \mid \mathfrak{F}_0\right)
\end{equation}

*where*
\begin{align*}
\mu &= -\xi \log \epsilon,\\
v_0 &= -\xi \log e_0.
\end{align*}

Denote $e_0^*$, $e_1^*$, $\hat{\lambda}^*$ as the solution to the above optimization problem. The implied solution for the probablity distortion is:

\begin{equation}
M_1^* = \frac{\exp \left[-\frac{1}{\xi}g(X_1)+\hat{\lambda}^*(Z_0)\cdot f(X_1)\right]e_1^*}{\epsilon^*e_0^*}
\end{equation}

In [11]:
def objective(λ):
    selector = pd_lag_indicator[:,state-1]
    term_1 = -g[selector]/ξ
    term_2 = f[:,(state-1)*4:state*4][selector]@λ
    term_3 = np.log(pd_indicator[selector]@e)
    x = term_1 + term_2 + term_3
    # use "max trick" to improve accuracy
    a = x.max()
    # log_E_exp(x)
    return np.log(np.sum(np.exp(x-a))) + a

def objective_gradient(λ):
    selector = pd_lag_indicator[:,state-1]
    temp1 = -g[selector]/ξ + f[:,(state-1)*4:state*4][selector]@λ + np.log(pd_indicator[selector]@e)
    temp2 = f[:,(state-1)*4:state*4][selector]*(np.exp(temp1.reshape((len(temp1),1)))/np.mean(np.exp(temp1)))
    temp3 = np.empty(temp2.shape[1])
    for i in range(temp2.shape[1]):
        temp3[i] = np.mean(temp2[:,i])
    return temp3

def min_objective():
    model = minimize(objective, 
                     np.ones(4), 
                     method='L-BFGS-B',
                     jac = objective_gradient,
                     tol=1e-10,
                     options={'maxiter': 1000})
    v = np.exp(model.fun)/np.sum(pd_lag_indicator[:,state-1])
    λ = model.x
    return v,λ

def iteration():
    # set global variables
    global state
    global e
    
    # initial error
    error = 1.
    # count times
    count = 0

    while error > 1e-10:
        if count == 0:
            # initial guess for e
            e = np.array([1,1,1])
            # placeholder for v
            v = np.zeros(3)   
            # placeholder for λ
            λ = np.zeros(12)
        for k in [1,2,3]:
            state = k
            v[state-1],λ[(state-1)*4:state*4] = min_objective()
        # update e and ϵ
        e_old = e
        ϵ = v[0]
        e = v/v[0]
        error = np.max(np.abs(e - e_old))
        count += 1
        
    return ϵ,e,λ,count

In [32]:
ξ = 10.

time_start = time.time() 
ϵ,e,λ,count = iteration()

print("--- %s seconds ---" % (round(time.time()-time_start,4)))
print("--- %s iterations ---" % count)
print("--- ϵ: %s ---" % ϵ)
print("--- e: %s ---" % e)
print("--- λ: %s ---" % λ)

--- 1.0462 seconds ---
--- 238 iterations ---
--- ϵ: 0.9713812958970629 ---
--- e: [1.         0.45242489 0.20311627] ---
--- λ: [ 1.63080641  0.36919359 -0.30577571 -0.73402231  1.84110182  0.15889818
  0.6526375  -5.89695635  3.12723834 -1.12723834 -3.44674047 -7.91157793] ---


In [33]:
# Calculate M
M = 1./ϵ * np.exp(-g/ξ+f@λ) * (pd_indicator@e) / (pd_lag_indicator@e)

# Check 1: E[M|state k] = 1
print("E[M|state 1] = %s " % np.mean(M[pd_lag_indicator[:,0]]))
print("E[M|state 2] = %s " % np.mean(M[pd_lag_indicator[:,1]]))
print("E[M|state 3] = %s " % np.mean(M[pd_lag_indicator[:,2]]))

E[M|state 1] = 0.9999999999953286 
E[M|state 2] = 0.9999999997949776 
E[M|state 3] = 0.9999999996663353 


In [34]:
# Calculate  conditional relative entropy
RE_1 = np.mean(M[pd_lag_indicator[:,0]]*np.log(M[pd_lag_indicator[:,0]]))
RE_2 = np.mean(M[pd_lag_indicator[:,1]]*np.log(M[pd_lag_indicator[:,1]]))
RE_3 = np.mean(M[pd_lag_indicator[:,2]]*np.log(M[pd_lag_indicator[:,2]]))

# Print conditional relative entropy
print("E[MlogM|state 1] = %s " % RE_1)
print("E[MlogM|state 2] = %s " % RE_2)
print("E[MlogM|state 3] = %s " % RE_3)

E[MlogM|state 1] = 0.011489543970283003 
E[MlogM|state 2] = 0.06470958916017636 
E[MlogM|state 3] = 0.16595891072934774 


In [35]:
# Calculate transition matrix under the distorted belief
P = np.zeros((3,3))
for i in [1,2,3]:
    for j in [1,2,3]:
        P[i-1,j-1] = np.mean(M[pd_lag_indicator[:,i-1]]*pd_indicator[pd_lag_indicator[:,i-1]][:,j-1])
# Print out the transition matrix
print("P_tilde:\n", P)
# P@np.ones(3).reshape(3,1)

P_tilde:
 [[0.97856179 0.02143821 0.        ]
 [0.08214167 0.88190482 0.03595351]
 [0.         0.17247155 0.82752845]]


In [36]:
# Calculate the stationary distribution
A = P.T - np.eye(3)
A[-1] = np.ones(3)
B = np.zeros(3)
B[-1] = 1.
π = np.linalg.solve(A, B)
# π_check = np.linalg.matrix_power(P, 500)
print("π = %s" % π)

π = [0.76022678 0.19841213 0.0413611 ]


In [37]:
# Calculate unconditional relative entropy
RE = np.array([RE_1,RE_2,RE_3]) @ π
print("E[MlogM] = %s" % RE)

E[MlogM] = 0.028438069060564265


In [42]:
# Implied moment bound, 1st approach: use \mu-\xi*RE
μ = - 1./ξ * np.log(ϵ)
moment_bound = μ - ξ*RE

print("E[Mg(X)] = %s" % moment_bound)

E[Mg(X)] = -0.2814770702010629


In [39]:
# Implied moment bound, 2nd approach: use E[Mg(X)|state]π
Mg_1 = np.mean(M[pd_lag_indicator[:,0]]*g[pd_lag_indicator[:,0]])
Mg_2 = np.mean(M[pd_lag_indicator[:,1]]*g[pd_lag_indicator[:,1]])
Mg_3 = np.mean(M[pd_lag_indicator[:,2]]*g[pd_lag_indicator[:,2]])
moment_bound_check = np.array([Mg_1,Mg_2,Mg_3]) @ π

In [40]:
moment_bound_check

0.005980572378843476

In [51]:
moment_bound_check - μ

0.003076951974263723

In [52]:
RE

0.028438069060564265